# Dummy Agent Library

Building a minimal agent from scratch — no framework, just Python.

**Reference:** [HF Agents Course — Unit 1: Dummy Agent Library](https://github.com/huggingface/agents-course/blob/main/units/en/unit1/dummy-agent-library.mdx)

---
### Setup
```
pip install huggingface_hub
```
Set your HF token as the environment variable `HF_TOKEN` (or in Colab → Secrets).

In [1]:
import os
from huggingface_hub import InferenceClient

## You need a token from https://hf.co/settings/tokens, ensure that you select 'read' as the token type. If you run this on Google Colab, you can set it up in the "settings" tab under "secrets". Make sure to call it "HF_TOKEN"
# HF_TOKEN = os.environ.get("HF_TOKEN")

client = InferenceClient(model="moonshotai/Kimi-K2.5")

**Kimi-K2.5** is developed by [Moonshot AI](https://www.moonshot.cn/), a Chinese AI research company. It is a large mixture-of-experts (MoE) model with strong instruction-following and reasoning capabilities. We use it here because:

- It is available for free on the HF Serverless Inference API with no local setup required
- It reliably follows the ReAct format specified in the system prompt
- It supports an optional extended-thinking mode (which we disable with `extra_body={"thinking": {"type": "disabled"}}` to keep outputs shorter and more predictable)

## 1. Serverless API — quick smoke test

In [3]:
output = client.chat.completions.create(
    messages=[
        {"role": "user", "content": "The capital of France is"},
    ],
    stream=False,
    max_tokens=1024,
    extra_body={'thinking': {'type': 'disabled'}},
)
print(output.choices[0].message.content)

Paris


SyntaxError: invalid syntax. Perhaps you forgot a comma? (4031191622.py, line 1)

## 1. Serverless API — quick smoke test

## 2. System prompt — tools + ReAct format

In [ ]:
SYSTEM_PROMPT = """Answer the following questions as best you can. \
You have access to the following tools:

get_weather: Get the current weather in a given location

The way you use the tools is by specifying a json blob.
Specifically, this json should have an `action` key (with the name of the tool to use)
and an `action_input` key (with the input to the tool going here).

The only values that should be in the "action" field are:
  get_weather: Get the current weather in a given location,
               args: {{"location": {{"type": "string"}}}}

example use:
  {{ "action": "get_weather", "action_input": {{"location": "New York"}} }}

ALWAYS use the following format:

Question: the input question you must answer
Thought: you should always think about one action to take. Only one action at a time.
Action:
```
$JSON_BLOB
```
Observation: the result of the action.
... (Thought/Action/Observation can repeat N times)

You must always end with:
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Now begin! Reminder to ALWAYS use the exact characters `Final Answer:` when responding.
"""

## 3. First call — spot the hallucination

In [ ]:
messages = [
    {"role": "system", "content": SYSTEM_PROMPT},
    {"role": "user",   "content": "What's the weather in London?"},
]

output = client.chat.completions.create(
    messages=messages,
    stream=False,
    max_tokens=200,
    extra_body={"thinking": {"type": "disabled"}},
)
print(output.choices[0].message.content)

## 4. Fix: stop before the model invents an Observation

In [ ]:
output = client.chat.completions.create(
    messages=messages,
    max_tokens=150,
    stop=["Observation:"],   # stop here so we can inject the real result
    extra_body={"thinking": {"type": "disabled"}},
)
partial = output.choices[0].message.content
print(partial)

## 5. Dummy tool

In [ ]:
def get_weather(location: str) -> str:
    """Return fake weather data for a location."""
    return f"the weather in {location} is sunny with low temperatures.\n"

print(get_weather("London"))

## 6. Inject real observation and resume

In [ ]:
messages_with_obs = [
    {"role": "system",    "content": SYSTEM_PROMPT},
    {"role": "user",      "content": "What's the weather in London?"},
    {"role": "assistant", "content": partial
                                     + "Observation:\n"
                                     + get_weather("London")},
]

final_output = client.chat.completions.create(
    messages=messages_with_obs,
    stream=False,
    max_tokens=200,
    extra_body={"thinking": {"type": "disabled"}},
)
print(final_output.choices[0].message.content)

## 7. Experiment — try a different city or add a second tool

Ideas:
- Change the city in the user message
- Add a `get_time(city)` tool and update the system prompt
- Ask a two-step question that requires both tools

In [ ]:
# Your experiments here
